# Install Hakai Timeseries Processing Package ( Google Colab)

In [ ]:
!pip install git+https://github.com/HakaiInstitute/process_ocean_timeseries.git

# Import the CTD timeseries method

In [ ]:
from process_ocean_data import process_ctd_timeseries
from process_ocean_data.tools import qc as qc

import ipywidgets as widgets
import xarray as xr
import numpy as np

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Retrieve Hakai Deployment Log

In [ ]:
# Retrieve Hakai Instrument Deployment Log 
instrument_log = process_ctd_timeseries.get_hakai_ctd_log()
instrument_log

# Download Data Locally To be Processed

In [ ]:
instrument_log = process_ctd_timeseries.download_raw_data(instrument_log)

In [ ]:
process_ctd_timeseries.get_ctd_qc_config()

In [ ]:
# Apply Standard Processing to the data
file_list = []
for index, row in instrument_log.iterrows():
    file_list +=[process_ctd_timeseries.process_data(row)]

In [ ]:
# Retrieve processed Files
file_list = [item for item in file_list if item]
l1_files = [item['l1'] for item in file_list]

# Manual Review (Not compatible with Google Colab)


In [ ]:
# Review one file at the time
file_selection = widgets.Select(
    options=l1_files,
    value=l1_files[0],
    description='File to review:'
)
file_selection

In [ ]:
# Load Selected Dataset
ds = xr.open_dataset(file_selection.value)
df = ds.to_dataframe().reset_index()


In [ ]:
# Generate a review flag from the QARTOD aggregate. If present the review flag will overwrite the QARTOD flag on a L2 data
flag_to_review = "qartod_aggregate"
for var in df.filter(like=flag_to_review).columns:
    df[var.replace(flag_to_review,'review_flag')] = df[var].astype(int).astype(str)

In [ ]:
# NOT COMPATIBLE WITH GOOGLE COLAB 
review_flag_suffix = "_review_flag"
qc.manual_qc_interface(df,
                       variable_list= ['TEMPS901','PSALST01', 'DOXYZZ01', 'CNDCST01'],
                       flags= {'GOOD':'1','UNKNOWN':'2','SUSPECT':'3','FAIL':'4','MISSING':'9'},
                       review_flag=review_flag_suffix)

In [ ]:
# Convert all flag variables to a standard type: {QARTOD: integer, Hakai: String}
for name,value in df.items():
    if review_flag_suffix in name:
        try:
            df[name] = df[name].astype('int8')
        except ValueError:
            df[name] = df[name].astype(str)
df               

# Generate QCed files
1. Level 1: add manual flag to a review column
1. Level 2: Drop Bad Data 


In [ ]:
# Seabird tends to have a fill value equal -9E-29. We'll replace that by np.nan

ds = old_ds.copy()
for name, value in ds.items():
    if '_FillValue' in value.encoding and 'missing_value' in value.encoding:
        value.encoding.pop('_FillValue')
for name, len in ds.dims.items():
    if '_FillValue' in ds[name].encoding and 'missing_value' in ds[name].encoding:
        ds[name].encoding.pop('_FillValue')

In [ ]:
# Level 1 Files
# Add selected flags back in the L1 file.
for var in df.filter(like=review_flag_suffix).columns:
    associated_variable = var.replace(review_flag_suffix,'')
    ds[var] = (ds[associated_variable].dims, df[var])
    
    if 'ancillary_variable' in ds[associated_variable].attrs:
        ds[associated_variable].attrs["ancillary_variable"] += ' '+var
    else:
        ds[associated_variable].attrs["ancillary_variable"] = var
        
# Save to NetCDF
l1_review_file = file_selection.value.rsplit('.')[0]+'_review.nc'
ds.to_netcdf(l1_review_file)

In [ ]:
# Level 2 Flag
ds_level2 = ds.copy()

consider_only=[1]
fillValue = np.nan
for name, value in ds_level2.items():
    if review_flag_suffix in name:
        related_variable = name.replace(review_flag_suffix,'')
        
        ds_level2[related_variable] = ds_level2[related_variable].where(ds_level2[name].isin(consider_only))
        